In [545]:
import numpy as np
import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
from qiskit_ionq import IonQProvider

from qiskit_aer import AerSimulator
# Import from Qiskit Aer noise module
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

provider = IonQProvider() # ADD API Token

In [546]:
#n = 23 # size of IonQ device which we are using
num_not_iterations = 10
waiting_time = 20
#qc = QuantumCircuit(n,n)
def xnot_noise_circuit(qc, n, num_not_iterations):
    for i in range(num_not_iterations):
        for j in range(n):
            qc.x(j)
            qc.x(j)
    return qc
def identity_noise_circuit(qc, n, waiting_time):
    for i in range(waiting_time):
        for j in range(n):
            qc.id(j)
    return qc

In [547]:
# initial should be a binary string from previous output
def noise_coloring(initial, num_not_iterations, waiting_time):
    n = len(initial)
    qc = QuantumCircuit(n,n)
    # initialize circuit
    for i in range(n):
        if int(initial[i]) == 1:
            qc.x(i)
        else:
            pass
    # initialize circuit
    xnot_noise_circuit(qc, n, num_not_iterations)
    identity_noise_circuit(qc, n, waiting_time)
    for i in range(n):
        qc.measure(i, i)
    return qc

In [548]:
# add simulated noise
# https://qiskit.org/documentation/tutorials/simulators/3_building_noise_models.html

# Example error probabilities
p_reset = 0.03
p_meas = 0.1
p_gate1 = 0.4

# QuantumError objects
error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
error_gate2 = error_gate1.tensor(error_gate1)

# Add errors to noise model
noise_bit_flip = NoiseModel()
noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
noise_bit_flip.add_all_qubit_quantum_error(error_meas, "measure")
noise_bit_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3", "id", "X"])
noise_bit_flip.add_all_qubit_quantum_error(error_gate2, ["cx"])

print(noise_bit_flip)

# Create noisy simulator backend
sim_noise = AerSimulator(noise_model=noise_bit_flip)

# Run and get counts
def result_with_noise(circuit):
    # Transpile circuit for noisy basis gates
    circuit_with_noise = transpile(circuit, sim_noise)

    return sim_noise.run(circuit_with_noise).result()


NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx', 'u1', 'u2', 'u3']
  Instructions with noise: ['u3', 'u2', 'cx', 'X', 'reset', 'id', 'u1', 'measure']
  All-qubits errors: ['reset', 'measure', 'u1', 'u2', 'u3', 'id', 'X', 'cx']


In [549]:
noise_coloring('01010101', num_not_iterations, waiting_time)

In [550]:
backend = provider.get_backend("ionq_simulator")

In [551]:
def noise_coloring_ionq(initial, num_not_iterations, waiting_time, device, sim_noise):
    qc = noise_coloring(initial, num_not_iterations, waiting_time)
    backend = 0
    if sim_noise == True:
        pass
    elif device == 'simulator':
        backend = provider.get_backend("ionq_simulator")
    elif device == 'device':
        backend = provider.get_backend("ionq_qpu")
        
    else:
        return "Error"

    if not sim_noise:
        job = backend.run(qc, shots = 1)
        result = job.result()
    else:
        result = result_with_noise(qc)

    counts = result.get_counts()
    measured_state_rev = counts.most_frequent()
    measured_state = measured_state_rev[::-1]
    return measured_state

In [552]:
# generate input qubit configuration

def bit_string(count, val):
    string = ""
    for _ in range(count):
        string += val
    return string

input_bits = bit_string(12, "1")

print(input_bits)

111111111111


In [553]:
input_bits = "111111111111"
input_bits = "110111100011"
input_bits = "100001011011"
input_bits = "100000011011"
input_bits = "100000000011"
# input_bits = "110100110010"
# input_bits = "110100110010"


measured_state = noise_coloring_ionq(input_bits, num_not_iterations, waiting_time, device = 'simulator', sim_noise = True)
print(measured_state)

100000000011


In [554]:
for bit in measured_state:
    print(bit)

1
0
0
0
0
0
0
0
0
0
1
1


In [555]:
for i in reversed(measured_state):
    print(i)

1
1
0
0
0
0
0
0
0
0
0
1
